In [1]:
import pandas as pd
import numpy as np
import scipy.linalg as la
import matplotlib.pyplot as plt
import sys
import os
sys.path.insert(0,'..')
from src.etl.etl import *
from src.scripts.scrape import *
import random
import time
import extcolors
import urllib.request
import urllib

# Data paparation
1. Get all the company logo's img url (ending with .png)
2. download all the img store it in the data logo img folder
3. every time download an image, get the rgb of the img
4. store the rgb in another dataframe


In [29]:
data_path= os.path.join(os.path.abspath(os.pardir),"data\\companies_200_500.txt")
with open(data_path,"r",encoding = "utf-8") as f:
    ranking = f.read().split("\n")
last_300 = ["https://craft.co/"+i for i in ranking]

In [59]:
data_path= os.path.join(os.path.abspath(os.pardir),"data\\companies_names_200_500.txt")
with open(data_path,"r",encoding = "utf-8") as f:
    names_ranking = f.read().split("\n")
names_ranking
len(names_ranking)

300

In [31]:
url_list = 300*[0]

In [32]:
class_name_for_logos = "summary__company-main-logo"
for i in range(0,300):
    
    url = last_300[i]
    company_name = ranking.loc[i]["companyName"]
    d = initialize_driver(url)
    d.get(url)
    time.sleep(1)
    try:
        logo_url = d.find_elements_by_class_name(class_name_for_logos)[0].get_attribute("src")
    except:
        print("Couldn't find logo for this company")
        logo_url = None
    kill_driver(d)
    pause_second = random.randint(1,5)
    time.sleep(pause_second)
    url_list[i] = logo_url

Couldn't find logo for this company
Couldn't find logo for this company
Couldn't find logo for this company
Couldn't find logo for this company
Couldn't find logo for this company
Couldn't find logo for this company
Couldn't find logo for this company
Couldn't find logo for this company
Couldn't find logo for this company
Couldn't find logo for this company
Couldn't find logo for this company
Couldn't find logo for this company
Couldn't find logo for this company


In [45]:
url_list

['https://uploads1.craft.co/uploads/company/logo/556xx/55626/normal_be147f4bad19df06.jpg',
 'https://uploads2.craft.co/uploads/company/logo/1184xx/118496/normal_ross_logo_fb.jpg',
 'https://uploads2.craft.co/uploads/company/logo/20xx/2056/normal_dgZGn-DI.jpeg',
 'https://uploads4.craft.co/uploads/company/logo/1150xx/115011/normal_86ed27f1c86de91f.png',
 'https://uploads5.craft.co/uploads/company/logo/19xx/1974/normal_5cfba4aafd0d5ff7.png',
 'https://uploads2.craft.co/uploads/company/logo/527xx/52766/normal_38f5ddb2a8753e68.png',
 'https://uploads2.craft.co/uploads/company/logo/200xx/20076/normal_3c16f89ee7a65b06.jpg',
 'https://uploads2.craft.co/uploads/company/logo/1274xx/127403/normal_51a1a24c43046056.png',
 'https://uploads2.craft.co/uploads/company/logo/1205xx/120551/normal_09ab0fc59b78378d.jpg',
 'https://uploads1.craft.co/uploads/company/logo/19xx/1917/normal_c8742248a9919dc5.jpg',
 'https://uploads2.craft.co/uploads/company/logo/18xx/1849/normal_b2ae4a0f88cdd2d1.png',
 'https://

In [62]:
ranking_300_last = pd.DataFrame({"companyName":names_ranking,"Ranking Actual":range(201,501),"craftWebsite":last_300,"imgUrl":url_list})

In [63]:

data_path_img_url= os.path.join(os.path.abspath(os.pardir),"data/img_urls_200_500.txt")
ranking_300_last.to_csv(data_path_img_url)


In [2]:
data_path_img_url= os.path.join(os.path.abspath(os.pardir),"data/img_urls_200_500.txt")
img_urls = pd.read_csv(data_path_img_url)
img_urls.head()

,Unnamed: 0,companyName,Ranking Actual,craftWebsite,imgUrl
0,0,Principal Financial,201,https://craft.co/principal-financial-group,https://uploads1.craft.co/uploads/company/logo...
1,1,Ross Stores,202,https://craft.co/ross-stores,https://uploads2.craft.co/uploads/company/logo...
2,2,Colgate-Palmolive,203,https://craft.co/colgate-palmolive,https://uploads2.craft.co/uploads/company/logo...
3,3,American Electric Power,204,https://craft.co/american-electric-power,https://uploads4.craft.co/uploads/company/logo...
4,4,Nordstrom,205,https://craft.co/nordstrom,https://uploads5.craft.co/uploads/company/logo...


In [6]:
len(img_urls)

300

## Download Images


In [3]:
imgFolder = os.path.join(os.path.abspath(os.pardir),"data/logos/")

In [4]:
def download_image_from_url(url,folder_location,name):
    '''
    url: the destination url of the image
    name: filename of the image to be stored, before the dot jpg
    folder_location: local file path of the image to be stored. e.g. data/test_data/
    '''
    extension = url.split(".")[-1]
    filename = folder_location+name+"."+extension
    
    urllib.request.urlretrieve(url, filename)
    return filename

In [7]:
colors_list = [0] * 300
for i in range(0,len(img_urls)):
    
    imgUrl = img_urls.loc[i]["imgUrl"]
    if not isinstance(imgUrl,str):
        print(imgUrl)
        continue
    ranking = i+200
    fileName = str(ranking).zfill(3)
    filename = download_image_from_url(imgUrl,imgFolder,fileName)
    colors, pixel_count = extcolors.extract_from_path(filename)
    colors_list[i] = colors

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


In [8]:
img_urls["color_rgb"] = colors_list

In [9]:
img_urls["numberOfColor"] = img_urls["color_rgb"].apply(lambda x:len(x) if (isinstance(x,list)) else 0)

In [10]:
rgb_file = os.path.join(os.path.abspath(os.pardir),"data/rgb_200_500.csv")
img_urls.drop("Unnamed: 0",axis=1).to_csv(rgb_file)

In [11]:
json_path = os.path.join(os.path.abspath(os.pardir),"config/credentials/credentials.json")
write_to_worksheet(rgb_file,json_path,"RGB of 200_500 Logos companies Fortune 500",["caiwei.wang.25@gmail.com"])

Created new spreadsheet:RGB of 200_500 Logos companies Fortune 500 From:/home/caw062/balance_scrape/data/rgb_200_500.csv
Sent to caiwei.wang.25@gmail.com
